In [1]:
from __future__ import print_function
import os
import warnings
warnings.filterwarnings('ignore')

import time
import torch
import shutil
import argparse
from m2det import build_net
import torch.utils.data as data
import torch.backends.cudnn as cudnn
from layers.functions import PriorBox
from data import detection_collate
from configs.CC import Config
from utils.core import *

In [2]:
torch.cuda.set_device(1)

In [3]:
args = []
config = 'configs/m2det512_resnet101.py'

dataset_prefix = 'VOC'
ngpu = 1
resume_net = None
resume_epoch = 0
tensorboard = False


print_info('----------------------------------------------------------------------\n'
           '|                       M2Det Training Program                       |\n'
           '----------------------------------------------------------------------',['yellow','bold'])

logger = set_logger(tensorboard)
global cfg
cfg = Config.fromfile(config)
net = build_net('train', 
                size = cfg.model.input_size, # Only 320, 512, 704 and 800 are supported
                config = cfg.model.m2det_config)
init_net(net, cfg, resume_net) # init the network with pretrained weights or resumed weights
if ngpu>1:
    net = torch.nn.DataParallel(net)
if cfg.train_cfg.cuda:
    net.cuda()
    cudnn.benchmark = True

optimizer = set_optimizer(net, cfg)
criterion = set_criterion(cfg)
priorbox = PriorBox(anchors(cfg))

with torch.no_grad():
    priors = priorbox.forward()
    if cfg.train_cfg.cuda:
        priors = priors.cuda()

net.train()
epoch = resume_epoch
print_info('===> Loading Dataset...',['yellow','bold'])
dataset = get_dataloader(cfg, dataset_prefix, 'train_sets')
epoch_size = len(dataset) // (cfg.train_cfg.per_batch_size * ngpu)
max_iter = cfg.train_cfg.step_lr[dataset_prefix][-1] * epoch_size
stepvalues = [_*epoch_size for _ in cfg.train_cfg.step_lr[dataset_prefix][:-1]]
print_info('===> Training M2Det on ' + dataset_prefix, ['yellow','bold'])
step_index = 0
if resume_epoch > 0:
    start_iter = resume_epoch * epoch_size
else:
    start_iter = 0
for iteration in range(start_iter, max_iter):
    if iteration % epoch_size == 0:
        batch_iterator = iter(data.DataLoader(dataset, 
                                              cfg.train_cfg.per_batch_size * ngpu, 
                                              shuffle=True, 
                                              num_workers=cfg.train_cfg.num_workers, 
                                              collate_fn=detection_collate))
        if epoch % cfg.model.save_eposhs == 0:
            save_checkpoint(net, cfg, final=False, datasetname = dataset_prefix, epoch=epoch)
        epoch += 1
    load_t0 = time.time()
    if iteration in stepvalues:
        step_index += 1
    lr = adjust_learning_rate(optimizer, cfg.train_cfg.gamma, epoch, step_index, iteration, epoch_size, cfg)
    images, targets = next(batch_iterator)
    if cfg.train_cfg.cuda:
        images = images.cuda()
        targets = [anno.cuda() for anno in targets]
    out = net(images)
    optimizer.zero_grad()
    loss_l, loss_c = criterion(out, priors, targets)
    loss = loss_l + loss_c
#     write_logger({'loc_loss':loss_l.item(),
#                   'conf_loss':loss_c.item(),
#                   'loss':loss.item()},logger,iteration,status=tensorboard)
    loss.backward()
    optimizer.step()
    load_t1 = time.time()
    print_train_log(iteration, cfg.train_cfg.print_epochs,
                        [time.ctime(),epoch,iteration%epoch_size,epoch_size,iteration,loss_l.item(),loss_c.item(),load_t1-load_t0,lr])
check_point_path = save_checkpoint(net, cfg, final=True, datasetname=dataset_prefix,epoch=-1)

----------------------------------------------------------------------
|                       M2Det Training Program                       |
----------------------------------------------------------------------
===> Constructing M2Det model
Initializing weights for [tums, reduce, up_reduce, leach, loc, conf]...
===> Loading Dataset...
===> Training M2Det on VOC
Time:Mon Jun 24 15:47:32 2019||Epoch:1||EpochIter:0/801||Iter:0||Loss_L:2.7401||Loss_C:13.9496||Batch_Time:7.8660||LR:0.0000010
Time:Mon Jun 24 15:47:37 2019||Epoch:1||EpochIter:10/801||Iter:10||Loss_L:3.0277||Loss_C:13.7426||Batch_Time:0.5012||LR:0.0000110
Time:Mon Jun 24 15:47:42 2019||Epoch:1||EpochIter:20/801||Iter:20||Loss_L:3.3323||Loss_C:13.8064||Batch_Time:0.4990||LR:0.0000210
Time:Mon Jun 24 15:47:47 2019||Epoch:1||EpochIter:30/801||Iter:30||Loss_L:2.8660||Loss_C:13.1541||Batch_Time:0.4999||LR:0.0000310
Time:Mon Jun 24 15:47:52 2019||Epoch:1||EpochIter:40/801||Iter:40||Loss_L:3.0067||Loss_C:12.5352||Batch_Time:0.4995|

Time:Mon Jun 24 15:52:14 2019||Epoch:1||EpochIter:560/801||Iter:560||Loss_L:2.9133||Loss_C:6.3172||Batch_Time:0.5006||LR:0.0005602
Time:Mon Jun 24 15:52:19 2019||Epoch:1||EpochIter:570/801||Iter:570||Loss_L:5.2251||Loss_C:39.5780||Batch_Time:0.5000||LR:0.0005701
Time:Mon Jun 24 15:52:24 2019||Epoch:1||EpochIter:580/801||Iter:580||Loss_L:3.3208||Loss_C:7.5431||Batch_Time:0.5020||LR:0.0005801
Time:Mon Jun 24 15:52:29 2019||Epoch:1||EpochIter:590/801||Iter:590||Loss_L:3.9181||Loss_C:7.5185||Batch_Time:0.5018||LR:0.0005901
Time:Mon Jun 24 15:52:34 2019||Epoch:1||EpochIter:600/801||Iter:600||Loss_L:5.9785||Loss_C:8.1888||Batch_Time:0.5014||LR:0.0006001
Time:Mon Jun 24 15:52:39 2019||Epoch:1||EpochIter:610/801||Iter:610||Loss_L:3.5215||Loss_C:7.8734||Batch_Time:0.5018||LR:0.0006101
Time:Mon Jun 24 15:52:44 2019||Epoch:1||EpochIter:620/801||Iter:620||Loss_L:3.2068||Loss_C:3.6325||Batch_Time:0.5009||LR:0.0006201
Time:Mon Jun 24 15:52:49 2019||Epoch:1||EpochIter:630/801||Iter:630||Loss_L:3.1404

Time:Mon Jun 24 15:57:11 2019||Epoch:2||EpochIter:349/801||Iter:1150||Loss_L:3.4369||Loss_C:2.0765||Batch_Time:0.5022||LR:0.0011493
Time:Mon Jun 24 15:57:16 2019||Epoch:2||EpochIter:359/801||Iter:1160||Loss_L:3.6860||Loss_C:2.8273||Batch_Time:0.5009||LR:0.0011593
Time:Mon Jun 24 15:57:22 2019||Epoch:2||EpochIter:369/801||Iter:1170||Loss_L:4.7909||Loss_C:1.7779||Batch_Time:0.5019||LR:0.0011692
Time:Mon Jun 24 15:57:27 2019||Epoch:2||EpochIter:379/801||Iter:1180||Loss_L:4.1802||Loss_C:2.5360||Batch_Time:0.5009||LR:0.0011792
Time:Mon Jun 24 15:57:32 2019||Epoch:2||EpochIter:389/801||Iter:1190||Loss_L:3.4249||Loss_C:3.4414||Batch_Time:0.4996||LR:0.0011892
Time:Mon Jun 24 15:57:37 2019||Epoch:2||EpochIter:399/801||Iter:1200||Loss_L:4.7938||Loss_C:2.4442||Batch_Time:0.5018||LR:0.0011992
Time:Mon Jun 24 15:57:42 2019||Epoch:2||EpochIter:409/801||Iter:1210||Loss_L:3.9323||Loss_C:3.0307||Batch_Time:0.5023||LR:0.0012092
Time:Mon Jun 24 15:57:47 2019||Epoch:2||EpochIter:419/801||Iter:1220||Loss_L

Time:Mon Jun 24 16:02:09 2019||Epoch:3||EpochIter:138/801||Iter:1740||Loss_L:4.1251||Loss_C:2.7824||Batch_Time:0.5013||LR:0.0017384
Time:Mon Jun 24 16:02:14 2019||Epoch:3||EpochIter:148/801||Iter:1750||Loss_L:3.3513||Loss_C:1.9455||Batch_Time:0.5010||LR:0.0017484
Time:Mon Jun 24 16:02:19 2019||Epoch:3||EpochIter:158/801||Iter:1760||Loss_L:4.2635||Loss_C:2.6565||Batch_Time:0.5012||LR:0.0017584
Time:Mon Jun 24 16:02:24 2019||Epoch:3||EpochIter:168/801||Iter:1770||Loss_L:3.8102||Loss_C:4.0342||Batch_Time:0.5033||LR:0.0017683
Time:Mon Jun 24 16:02:29 2019||Epoch:3||EpochIter:178/801||Iter:1780||Loss_L:2.6866||Loss_C:3.1499||Batch_Time:0.5003||LR:0.0017783
Time:Mon Jun 24 16:02:34 2019||Epoch:3||EpochIter:188/801||Iter:1790||Loss_L:4.9986||Loss_C:3.0489||Batch_Time:0.4990||LR:0.0017883
Time:Mon Jun 24 16:02:39 2019||Epoch:3||EpochIter:198/801||Iter:1800||Loss_L:3.4989||Loss_C:3.7192||Batch_Time:0.5017||LR:0.0017983
Time:Mon Jun 24 16:02:44 2019||Epoch:3||EpochIter:208/801||Iter:1810||Loss_L

Time:Mon Jun 24 16:07:06 2019||Epoch:3||EpochIter:728/801||Iter:2330||Loss_L:2.5993||Loss_C:3.0777||Batch_Time:0.5009||LR:0.0023275
Time:Mon Jun 24 16:07:11 2019||Epoch:3||EpochIter:738/801||Iter:2340||Loss_L:3.2078||Loss_C:2.7767||Batch_Time:0.5017||LR:0.0023375
Time:Mon Jun 24 16:07:16 2019||Epoch:3||EpochIter:748/801||Iter:2350||Loss_L:3.0331||Loss_C:2.2215||Batch_Time:0.5010||LR:0.0023475
Time:Mon Jun 24 16:07:21 2019||Epoch:3||EpochIter:758/801||Iter:2360||Loss_L:3.0936||Loss_C:1.7656||Batch_Time:0.5026||LR:0.0023575
Time:Mon Jun 24 16:07:26 2019||Epoch:3||EpochIter:768/801||Iter:2370||Loss_L:2.6452||Loss_C:1.8385||Batch_Time:0.5003||LR:0.0023674
Time:Mon Jun 24 16:07:31 2019||Epoch:3||EpochIter:778/801||Iter:2380||Loss_L:2.6197||Loss_C:2.7260||Batch_Time:0.5014||LR:0.0023774
Time:Mon Jun 24 16:07:36 2019||Epoch:3||EpochIter:788/801||Iter:2390||Loss_L:2.2702||Loss_C:2.5483||Batch_Time:0.4994||LR:0.0023874
Time:Mon Jun 24 16:07:41 2019||Epoch:3||EpochIter:798/801||Iter:2400||Loss_L

...

Time:Tue Jun 25 08:20:11 2019||Epoch:147||EpochIter:794/801||Iter:117740||Loss_L:0.7840||Loss_C:2.8568||Batch_Time:0.5071||LR:0.0000400
Time:Tue Jun 25 08:20:16 2019||Epoch:148||EpochIter:3/801||Iter:117750||Loss_L:0.8596||Loss_C:2.1711||Batch_Time:0.5087||LR:0.0000400
Time:Tue Jun 25 08:20:21 2019||Epoch:148||EpochIter:13/801||Iter:117760||Loss_L:0.9434||Loss_C:2.2718||Batch_Time:0.5029||LR:0.0000400
Time:Tue Jun 25 08:20:26 2019||Epoch:148||EpochIter:23/801||Iter:117770||Loss_L:1.5079||Loss_C:3.2871||Batch_Time:0.5053||LR:0.0000400
Time:Tue Jun 25 08:20:31 2019||Epoch:148||EpochIter:33/801||Iter:117780||Loss_L:1.1354||Loss_C:3.4205||Batch_Time:0.5030||LR:0.0000400
Time:Tue Jun 25 08:20:36 2019||Epoch:148||EpochIter:43/801||Iter:117790||Loss_L:1.5619||Loss_C:2.9634||Batch_Time:0.5022||LR:0.0000400
Time:Tue Jun 25 08:20:41 2019||Epoch:148||EpochIter:53/801||Iter:117800||Loss_L:0.7179||Loss_C:3.1379||Batch_Time:0.5050||LR:0.0000400
Time:Tue Jun 25 08:20:46 2019||Epoch:148||EpochIter:63/

Time:Tue Jun 25 08:25:00 2019||Epoch:148||EpochIter:563/801||Iter:118310||Loss_L:0.7493||Loss_C:3.3147||Batch_Time:0.5057||LR:0.0000400
Time:Tue Jun 25 08:25:05 2019||Epoch:148||EpochIter:573/801||Iter:118320||Loss_L:1.2013||Loss_C:3.3802||Batch_Time:0.5069||LR:0.0000400
Time:Tue Jun 25 08:25:10 2019||Epoch:148||EpochIter:583/801||Iter:118330||Loss_L:1.3280||Loss_C:4.0192||Batch_Time:0.5034||LR:0.0000400
Time:Tue Jun 25 08:25:15 2019||Epoch:148||EpochIter:593/801||Iter:118340||Loss_L:0.2136||Loss_C:1.8965||Batch_Time:0.5034||LR:0.0000400
Time:Tue Jun 25 08:25:20 2019||Epoch:148||EpochIter:603/801||Iter:118350||Loss_L:0.8381||Loss_C:2.1022||Batch_Time:0.5043||LR:0.0000400
Time:Tue Jun 25 08:25:25 2019||Epoch:148||EpochIter:613/801||Iter:118360||Loss_L:0.5167||Loss_C:2.8221||Batch_Time:0.5058||LR:0.0000400
Time:Tue Jun 25 08:25:30 2019||Epoch:148||EpochIter:623/801||Iter:118370||Loss_L:0.9520||Loss_C:2.9055||Batch_Time:0.5050||LR:0.0000400
Time:Tue Jun 25 08:25:35 2019||Epoch:148||EpochI

Time:Tue Jun 25 08:29:49 2019||Epoch:149||EpochIter:332/801||Iter:118880||Loss_L:1.1195||Loss_C:3.7138||Batch_Time:0.5034||LR:0.0000400
Time:Tue Jun 25 08:29:54 2019||Epoch:149||EpochIter:342/801||Iter:118890||Loss_L:0.3514||Loss_C:2.5230||Batch_Time:0.5049||LR:0.0000400
Time:Tue Jun 25 08:29:59 2019||Epoch:149||EpochIter:352/801||Iter:118900||Loss_L:0.8475||Loss_C:3.2906||Batch_Time:0.5045||LR:0.0000400
Time:Tue Jun 25 08:30:04 2019||Epoch:149||EpochIter:362/801||Iter:118910||Loss_L:1.8576||Loss_C:2.6840||Batch_Time:0.5055||LR:0.0000400
Time:Tue Jun 25 08:30:09 2019||Epoch:149||EpochIter:372/801||Iter:118920||Loss_L:1.0466||Loss_C:3.5674||Batch_Time:0.5040||LR:0.0000400
Time:Tue Jun 25 08:30:14 2019||Epoch:149||EpochIter:382/801||Iter:118930||Loss_L:1.3492||Loss_C:3.6560||Batch_Time:0.5041||LR:0.0000400
Time:Tue Jun 25 08:30:19 2019||Epoch:149||EpochIter:392/801||Iter:118940||Loss_L:0.6110||Loss_C:2.2251||Batch_Time:0.5065||LR:0.0000400
Time:Tue Jun 25 08:30:24 2019||Epoch:149||EpochI

Time:Tue Jun 25 08:34:38 2019||Epoch:150||EpochIter:101/801||Iter:119450||Loss_L:1.4072||Loss_C:4.2939||Batch_Time:0.5049||LR:0.0000400
Time:Tue Jun 25 08:34:43 2019||Epoch:150||EpochIter:111/801||Iter:119460||Loss_L:1.3395||Loss_C:5.8065||Batch_Time:0.5073||LR:0.0000400
Time:Tue Jun 25 08:34:48 2019||Epoch:150||EpochIter:121/801||Iter:119470||Loss_L:0.7830||Loss_C:2.4093||Batch_Time:0.5052||LR:0.0000400
Time:Tue Jun 25 08:34:53 2019||Epoch:150||EpochIter:131/801||Iter:119480||Loss_L:0.6282||Loss_C:2.3520||Batch_Time:0.5057||LR:0.0000400
Time:Tue Jun 25 08:34:58 2019||Epoch:150||EpochIter:141/801||Iter:119490||Loss_L:0.9575||Loss_C:2.9147||Batch_Time:0.5056||LR:0.0000400
Time:Tue Jun 25 08:35:03 2019||Epoch:150||EpochIter:151/801||Iter:119500||Loss_L:0.2844||Loss_C:3.2446||Batch_Time:0.5037||LR:0.0000400
Time:Tue Jun 25 08:35:08 2019||Epoch:150||EpochIter:161/801||Iter:119510||Loss_L:0.7090||Loss_C:2.6116||Batch_Time:0.5046||LR:0.0000400
Time:Tue Jun 25 08:35:13 2019||Epoch:150||EpochI

Time:Tue Jun 25 08:39:26 2019||Epoch:150||EpochIter:671/801||Iter:120020||Loss_L:1.3487||Loss_C:3.6993||Batch_Time:0.5053||LR:0.0000400
Time:Tue Jun 25 08:39:32 2019||Epoch:150||EpochIter:681/801||Iter:120030||Loss_L:0.5414||Loss_C:2.4475||Batch_Time:0.5047||LR:0.0000400
Time:Tue Jun 25 08:39:37 2019||Epoch:150||EpochIter:691/801||Iter:120040||Loss_L:0.6557||Loss_C:3.1766||Batch_Time:0.5049||LR:0.0000400
Time:Tue Jun 25 08:39:42 2019||Epoch:150||EpochIter:701/801||Iter:120050||Loss_L:0.7782||Loss_C:2.7730||Batch_Time:0.5034||LR:0.0000400
Time:Tue Jun 25 08:39:47 2019||Epoch:150||EpochIter:711/801||Iter:120060||Loss_L:0.5426||Loss_C:2.2683||Batch_Time:0.5032||LR:0.0000400
Time:Tue Jun 25 08:39:52 2019||Epoch:150||EpochIter:721/801||Iter:120070||Loss_L:1.4035||Loss_C:3.1335||Batch_Time:0.5029||LR:0.0000400
Time:Tue Jun 25 08:39:57 2019||Epoch:150||EpochIter:731/801||Iter:120080||Loss_L:1.0734||Loss_C:2.5128||Batch_Time:0.5058||LR:0.0000400
Time:Tue Jun 25 08:40:02 2019||Epoch:150||EpochI

In [4]:
print('done')

done


In [5]:
check_point_path

'weights/Final_M2Det_VOC_size512_netresnet101.pth'